In [ ]:
import cv2
from glob import glob
import numpy as np

In [ ]:
noncar_supfolder = './data/non-vehicles/non-vehicles/'
noncar_img_list = sorted(glob(noncar_supfolder +'*/*.png'))

car_supfolder = './data/vehicles/vehicles/'
car_img_list = sorted(glob(car_supfolder +'*/*.png'))

print('Number of vehicle images:', len(car_img_list))
print('Number of non-vehicle images:', len(noncar_img_list))

In [ ]:
def filelist2array(filelist):
    out = []
    for fn in filelist:
        out.append(cv2.imread(fn))
    return np.array(out, dtype=np.uint8)

bags = 20
car_imgs = filelist2array(car_img_list[:len(car_img_list)//bags*bags])
noncar_imgs = filelist2array(noncar_img_list[:len(noncar_img_list)//bags*bags])

In [ ]:
car_imgs = car_imgs.reshape(-1,bags,*car_imgs.shape[-3:])
noncar_imgs = noncar_imgs.reshape(-1,bags,*noncar_imgs.shape[-3:])

In [ ]:
from sklearn.model_selection import train_test_split
car_train, car_test = train_test_split(car_imgs, test_size=0.2)
noncar_train, noncar_test = train_test_split(noncar_imgs, test_size=0.2)

car_train, car_val = train_test_split(car_train, test_size=0.25)
noncar_train, noncar_val = train_test_split(noncar_train, test_size=0.25)

In [ ]:
car_train = car_train.reshape(-1,*car_train.shape[-3:])
car_val = car_val.reshape(-1,*car_val.shape[-3:])
car_test = car_test.reshape(-1,*car_test.shape[-3:])

noncar_train = noncar_train.reshape(-1,*noncar_train.shape[-3:])
noncar_val = noncar_val.reshape(-1,*noncar_val.shape[-3:])
noncar_test = noncar_test.reshape(-1,*noncar_test.shape[-3:])

In [ ]:
from sklearn.utils import resample

def jitter_crop(img):
    top = np.random.randint(0,64/4)
    left = np.random.randint(0,64/4)
    height = np.random.randint(64/2,64)
    width = np.random.randint(64/2,64)
    bottom = min(top+height, 64)
    right = min(left+width, 64)
    return cv2.resize(img[top:bottom, left:right], (64,64))

def augment_data(imgs, n):
    return [jitter_crop(img) for img in resample(imgs, n_samples=n)]

In [ ]:
# car_train = np.vstack([car_train, augment_data(car_train, 2*len(car_train))])
# noncar_train = np.vstack([noncar_train, augment_data(noncar_train, 2*len(car_train))])

In [ ]:
np.random.randint?

In [ ]:
img_shape = car_train[0].shape
img_shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
RGB = lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(RGB(car_train[6]))

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

def histogram_equalization(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    yuv[:,:,0] = clahe.apply(yuv[:,:,0])
    return cv2.cvtColor(img, cv2.COLOR_YUV2BGR)

In [ ]:
def bin_spatial(img, cspace=None, size=(16, 16)):
    # Convert image to new color space (if specified)
    img = cv2.resize(img, size, cv2.INTER_AREA)
    if cspace is not None:
        img = cv2.cvtColor(img, cspace)
    return img.ravel()

def color_histogram(img,cspace=cv2.COLOR_BGR2HSV, bins=16):
    if cspace is not None:
        img = cv2.cvtColor(img, cspace)
    return np.hstack([np.histogram(np.sqrt(channel).ravel(),bins=bins,normed=True)[0] 
                      for channel in cv2.split(img)])
    

In [ ]:
# help(cv2.HOGDescriptor())

In [ ]:
import skimage.feature

def hog(img, orient=9, pix_per_cell=8, cell_per_block=2, vis=False, feature_vec=True):
    return skimage.feature.hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
               cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
               visualise=vis, feature_vector=feature_vec)

def hogcv(img, orient=32, pix_per_cell=7, cell_per_block=2):
    hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // pix_per_cell * pix_per_cell,
                                     img.shape[0] // pix_per_cell * pix_per_cell),
                            _blockSize=(cell_per_block * pix_per_cell,
                                       cell_per_block * pix_per_cell),
                            _blockStride=(pix_per_cell, pix_per_cell),
                            _cellSize=(pix_per_cell, pix_per_cell),
                            _nbins=orient)
    return hog.compute(img).ravel()
                            

In [ ]:
def extract_simple_features(img):
    return np.hstack([bin_spatial(img,cv2.COLOR_BGR2HSV)])

def extract_features(img):
    y,cr,cb = cv2.split(cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb))
    return np.hstack([hogcv(y), hogcv(cr), hogcv(cb), bin_spatial(img, cv2.COLOR_BGR2HSV)])

In [ ]:
from tqdm import tqdm

simple_features_car_train = [extract_simple_features(img) for img in tqdm(car_train)]
simple_features_noncar_train = [extract_simple_features(img) for img in tqdm(noncar_train)]
simple_features_train = np.array(simple_features_car_train + simple_features_noncar_train)

features_car_train = [extract_features(img) for img in tqdm(car_train)]
features_noncar_train = [extract_features(img) for img in tqdm(noncar_train)]
features_train = np.array(features_car_train + features_noncar_train)

In [ ]:
print(np.array(simple_features_car_train).shape)
np.array(features_car_train).shape

In [ ]:
labels_train = [1]*len(features_car_train) + [0]*len(features_noncar_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler().fit(features_train)
scaled_features_train = feature_scaler.transform(features_train)

simple_feature_scaler = StandardScaler().fit(simple_features_train)
scaled_simple_features_train = simple_feature_scaler.transform(simple_features_train)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score, accuracy_score
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV


pca1 = PCA(n_components=50)
pca1.fit(scaled_simple_features_train)

pca_simple_features_train = pca1.transform(scaled_simple_features_train)
# pca_simple_features_train = scaled_simple_features_train

# classifier = GaussianNB()
# classifier.fit(pca_simple_features_train, labels_train)

classifier = LinearSVC(tol=1e-12)
# classifier = SVC(tol=1e-8)
# classifier = DecisionTreeClassifier()
# classifier = AdaBoostClassifier()
# classifier = LogisticRegressionCV(tol=1e-8)
classifier.fit(pca_simple_features_train, labels_train)

simple_features_car_test = [extract_simple_features(img) for img in tqdm(car_test)]
simple_features_noncar_test = [extract_simple_features(img) for img in tqdm(noncar_test)]
simple_features_test = np.array(simple_features_car_test + simple_features_noncar_test)

labels_test = [1]*len(simple_features_car_test) + [0]*len(simple_features_noncar_test)

scaled_simple_features_test = simple_feature_scaler.transform(simple_features_test)
pca_simple_features_test = pca1.transform(simple_features_test)
# pca_simple_features_test = simple_features_test
pred_simple_test = classifier.predict(pca_simple_features_test)

print('Test Precision of GNB = ', round(precision_score(labels_test, pred_simple_test), 4))
print('Test Recall of GNB = ', round(recall_score(labels_test, pred_simple_test), 4))
print('Test Accuracy of GNB = ', round(accuracy_score(labels_test, pred_simple_test), 4))

In [ ]:
scores_test = classifier.decision_function(pca_simple_features_test)
from sklearn.metrics import roc_curve, precision_recall_curve

precision, recall, thresholds = precision_recall_curve(labels_test, scores_test)
plt.plot(precision, recall,)
plt.xlabel('Precision')
plt.ylabel('Recall')

In [ ]:
from sklearn.decomposition import PCA
# pca = PCA(n_components=1000)
# pca.fit(scaled_features_train)
# print(sum(pca.explained_variance_ratio_))

In [ ]:
# pca_features_train = pca.transform(scaled_features_train)
pca_features_train = scaled_features_train

In [ ]:
pca_features_train.shape

In [ ]:
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(pca_features_train, labels_train)

In [ ]:
def process(img):
    features = pack_features(img)
    scaled_features = feature_scaler.transform(features)
#     pca_features = pca.transform(scaled_features)
    pca_features = scaled_features
    return pca_features
    
features_car_test = [extract_features(img) for img in tqdm(car_test)]
features_noncar_test = [extract_features(img) for img in tqdm(noncar_test)]
features_test = np.array(features_car_test + features_noncar_test)

labels_test = [1]*len(features_car_test) + [0]*len(features_noncar_test)

scaled_features_test = feature_scaler.transform(features_test)
# pca_features_test = pca.transform(scaled_features_test)
pca_features_test = scaled_features_test

print('Test Precision of SVC = ', round(precision_score(labels_test, svc.predict(pca_features_test)), 4))
print('Test Recall of SVC = ', round(recall_score(labels_test, svc.predict(pca_features_test)), 4))
print('Test Accuracy of SVC = ', round(svc.score(pca_features_test, labels_test), 4))


In [ ]:
# Here is your draw_boxes function from the previous exercise
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    imcopy = np.copy(img)
    for bbox in bboxes:
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)        
    return imcopy

def slide_window(img_shape, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    x_start, x_stop = x_start_stop
    y_start, y_stop = x_start_stop
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img_shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img_shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = min(startx + xy_window[0], img_shape[1]-1)
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = min(starty + xy_window[1], img_shape[0]-1)
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


In [ ]:
img = cv2.imread('./test_images/test1.jpg')
img_shape = img.shape

windows192 = slide_window(img_shape, y_start_stop=[450, 600], 
                            xy_window=(192, 192), xy_overlap=(0.75, 0.75))
windows128 = slide_window(img_shape, y_start_stop=[450, 600], 
                            xy_window=(128, 128), xy_overlap=(0.75, 0.75))
windows96 = slide_window(img_shape, y_start_stop=[400, 520], 
                            xy_window=(96, 96), xy_overlap=(0.75, 0.75))
windows64 = slide_window(img_shape, y_start_stop=[400, 550], 
                            xy_window=(64, 64), xy_overlap=(0.75, 0.75))
windows = windows64 + windows96 + windows128 + windows192

In [ ]:
plt.imshow(RGB(draw_boxes(img, windows64)))

In [ ]:
def slide_window_extract_features(img, windows):
    feature1 = np.array(slide_window_color_histogram(img, windows))
    feature2 = np.array(slide_window_bin_spatial(img, windows))
    feature3 = np.array(slide_window_hog(img, windows))
    return np.hstack([feature1, feature2, feature3])

In [ ]:
def extract_features_from_windows(img, windows):
    features = []
    for (x0,y0), (x1,y1) in windows:
        patch = cv2.resize(img[y0:y1,x0:x1], (64,64))
        features.append(extract_features(patch))
    return features

def extract_simple_features_from_windows(img, windows):
    features = []
    for (x0,y0), (x1,y1) in windows:
        patch = cv2.resize(img[y0:y1,x0:x1], (64,64))
        features.append(extract_simple_features(patch))
    return features

def batch_extract_simple_features_from_windows(img, windows, binsize=(16,16)):
    binh, binw = binsize
    imgh, imgw = img.shape[:2]
    
    windows=np.int_(windows)
    wsize = np.diff(windows,axis=1).squeeze()
    unique_wsize = np.vstack({tuple(row) for row in wsize})
    
    patches = [None]*len(windows)
    
    for winw, winh in unique_wsize:
        rs_img = cv2.resize(img, (binw*imgw//winw+1,binh*imgh//winh+1))            
        idx, = np.where((wsize[:,0]==winh) &  (wsize[:,1]==winw))        
        topleft = windows[idx][:,0,:]*[binw,binh]//[winw,winh]
        for (x, y), i in zip(topleft, range(len(idx))):
            patches[idx[i]] = rs_img[y:y+binh,x:x+binw]

    return np.array(patches).reshape(len(windows),-1)

In [ ]:
def detect_veh(img):
    features = extract_features_from_windows(img, windows)
#     idx, = np.where(svc.predict(pca.transform(feature_scaler.transform(features)))>0.5)
    idx, = np.where(svc.predict(feature_scaler.transform(features))>0.5)
    return [windows[i] for i in idx]

def simple_detect_veh(img):
    features = batch_extract_simple_features_from_windows(img, windows)
#     scores = classifier.predict(simple_feature_scaler.transform(features))
    scaled_simple_features = simple_feature_scaler.transform(features)
#     pca_simple_features = pca1.transform(scaled_simple_features)
    pca_simple_features = scaled_simple_features
    scores = classifier.predict(pca_simple_features)
    idx, = np.where(scores>0.5)
    return draw_boxes(img, [windows[i] for i in idx])


In [ ]:
def add_heat(img_shape, windows):
    heatmap = np.zeros(img_shape[:2], dtype=int)
    for (x0,y0), (x1,y1) in windows:
        heatmap[y0:y1, x0:x1] += 1
    return heatmap

In [ ]:
%matplotlib inline
RGB = lambda x: cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
img = cv2.imread('test_images/test6.jpg')
detected_windows = detect_veh(img)
bboxed_img = draw_boxes(img, detected_windows)
heatmap = add_heat(img.shape, detected_windows)
plt.imshow(RGB(bboxed_img))
plt.figure()
plt.imshow(heatmap, cmap='hot')

In [ ]:
import scipy.ndimage.measurements
from queue import Queue

class History:
    def __init__(self):
        self.heatmaps = Queue(16)
        self.bboxes = []
        self.count = 0
        self.decay = 0.8
        
    def update(self, img):
        new_bboxes = detect_veh(img)
        new_heatmap = add_heat(img.shape, new_bboxes)
        if(self.heatmaps.full()): self.heatmaps.get()
        self.heatmaps.put(new_heatmap)
        intheatmaps = np.array(self.heatmaps.queue).sum(axis=0)
        labels = scipy.ndimage.measurements.label(intheatmaps>5)
        
        bboxes = []
        for veh in range(1, labels[1]+1):
            nonzeroy, nonzerox = (labels[0] == veh).nonzero()
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            if bbox[1][0] - bbox[0][0] > 50 and bbox[1][1]-bbox[0][1] > 50:
                bboxes.append(bbox)
        
        aaa = (intheatmaps*1.0/(intheatmaps.max()+0.01)*255).astype(np.uint8)
        aaa = cv2.resize(aaa, (aaa.shape[1]//4, aaa.shape[0]//4))
        aaa = cv2.cvtColor(aaa, cv2.COLOR_GRAY2BGR)
        img = draw_boxes(img, bboxes)
        img[:aaa.shape[0],:aaa.shape[1]] = aaa
        return img
        
        

In [ ]:
his = History()
his.update(cv2.imread('./test_images/test1.jpg'))
his.update(cv2.imread('./test_images/test1.jpg'))
his.update(cv2.imread('./test_images/test1.jpg'))
his.update(cv2.imread('./test_images/test1.jpg'))
plt.imshow(RGB(his.update(cv2.imread('./test_images/test1.jpg'))))

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

his = History()

rgb2bgr = lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2BGR)
input_name = './project_video.mp4'
output_name = './project_video_out.mp4'
clip1 = VideoFileClip(input_name)
white_clip = clip1.fl_image(lambda x: RGB(his.update(rgb2bgr(x)))) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output_name, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_name))

In [ ]:
cv2.HOGDescriptor?